In [ ]:
# 03_reporting — Cell 1
from __future__ import annotations

from pathlib import Path
import sys, os, json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# ---- Pfade & Konfiguration laden (aus 01) ----
BASE = Path.cwd().resolve().parents[0] if Path.cwd().name.lower()=="notebooks" else Path.cwd()
OUT  = BASE / "data" / "processed"
FIG  = BASE / "reports" / "figures"

CONFIG = json.loads((OUT / "project_config.json").read_text(encoding="utf-8"))
KANON  = CONFIG["kanon"]
FIG.mkdir(parents=True, exist_ok=True)

# ---- Daten laden ----
WIDE_PATH = OUT / "statista_harmonisiert_2021_2022_2024.csv"
LONG_PATH = OUT / "statista_long_2021_2022_2024.csv"

stat_pivot = pd.read_csv(WIDE_PATH)
stat_all   = pd.read_csv(LONG_PATH)

# Wide-Index setzen & konsistente Reihenfolge
stat_pivot = (stat_pivot
              .set_index("Kategorie")
              .reindex(KANON)
              .fillna(0.0))

YEARS = [c for c in stat_pivot.columns if str(c).isdigit()]
YEARS = sorted(map(int, YEARS))
print("Geladene Jahre:", YEARS)
display(stat_pivot.head(7))


In [ ]:
# 03_reporting — Cell 2
import matplotlib as mpl

plt.rcParams.update({
    "figure.dpi": 120, "savefig.dpi": 300,
    "font.size": 11, "axes.titlesize": 12, "axes.labelsize": 11,
    "axes.grid": True, "grid.alpha": 0.2,
    "axes.spines.top": False, "axes.spines.right": False
})

def percent_axis(ax, axis="y", decimals=0, limit=(0, 100)):
    fmt = mtick.PercentFormatter(xmax=100, decimals=decimals)
    if axis == "y":
        ax.set_ylim(*limit)
        ax.yaxis.set_major_formatter(fmt)
    else:
        ax.set_xlim(*limit)
        ax.xaxis.set_major_formatter(fmt)

def label_bars(ax, decimals=1, rotate=0):
    for c in ax.containers:
        ax.bar_label(c, fmt=f"%.{decimals}f %%", padding=2, fontsize=8, rotation=rotate)

def footer(source_note: str):
    plt.figtext(0.01, -0.04, source_note, ha="left", va="top", fontsize=9)

def save_fig(path: Path):
    plt.savefig(path, bbox_inches="tight", facecolor="white")
    plt.close()


In [ ]:
# 03_reporting — Cell 3
for yr in YEARS:
    values = stat_pivot[yr].sort_values(ascending=True)
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(values.index, values.values)
    # Prozentlabels rechts neben die Balken
    for rect, v in zip(bars, values.values):
        ax.text(rect.get_width() + 1.0, rect.get_y() + rect.get_height()/2,
                f"{v:.1f} %", va="center", fontsize=9)
    ax.set_title(f"Online-Kaufanteile nach Produktkategorien – {yr}")
    ax.set_xlabel("Anteil in %")
    percent_axis(ax, axis="x", decimals=0, limit=(0, 100))
    plt.tight_layout()
    footer(f"Quelle: Statista ({yr}). Einheit: Prozent. Hinweis: Harmonisierung gemäß Mapping; Teilkategorien konsolidiert.")
    save_fig(FIG / f"03_{yr}_niveau.png")

print("Niveauabbildungen exportiert →", FIG)


In [ ]:
# 03_reporting — Cell 4
fig, ax = plt.subplots(figsize=(12, 6))
(stat_pivot[YEARS]
 .plot(kind="bar", ax=ax, width=0.8))
label_bars(ax, decimals=1, rotate=90)
ax.set_title("Online-Kaufanteile nach Kategorien – Vergleich 2021 / 2022 / 2024")
ax.set_xlabel("Kategorie")
ax.set_ylabel("Anteil in %")
percent_axis(ax, axis="y", decimals=0, limit=(0, 100))
ax.legend(title="Jahr", ncols=len(YEARS), frameon=False, loc="upper left", bbox_to_anchor=(1.01, 1.0))
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
footer("Quelle: Statista (2021–2024). Einheit: Prozent. Hinweis: Harmonisierung gemäß Mapping; Teilkategorien konsolidiert.")
save_fig(FIG / "03_vergleich_2021_2022_2024.png")


In [ ]:
# 03_reporting — Cell 5
# Deltas 2021→2024, zusätzlich 2021→2022 und 2022→2024
delta = pd.DataFrame(index=stat_pivot.index)
delta["Δ 2021→2022"] = stat_pivot[2022] - stat_pivot[2021]
delta["Δ 2022→2024"] = stat_pivot[2024] - stat_pivot[2022]
delta["Δ 2021→2024"] = stat_pivot[2024] - stat_pivot[2021]
delta = delta.sort_values("Δ 2021→2024", ascending=False)

display(delta.style.format("{:.1f}"))

# Export als CSV für die Doku
DELTA_PATH = OUT / "statista_delta_2021_2022_2024.csv"
delta.to_csv(DELTA_PATH, encoding="utf-8")
print("Delta-Tabelle exportiert →", DELTA_PATH)


In [ ]:
# 03_reporting — Cell 6
order = delta.index  # nach Δ 2021→2024 sortiert
vals = delta["Δ 2021→2024"].reindex(order)

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(order, vals.values, color=["#4d8" if v>=0 else "#d66" for v in vals.values])
# Labels mit Vorzeichen
for rect, v in zip(bars, vals.values):
    ax.text((0.5 if v>=0 else -0.5) + v, rect.get_y()+rect.get_height()/2,
            f"{v:+.1f} pp", va="center", ha="left" if v>=0 else "right", fontsize=9)
ax.set_title("Veränderung der Online-Kaufanteile 2021→2024 (Prozentpunkte)")
ax.set_xlabel("Δ in Prozentpunkten")
# symmetrische Skala um 0 (keine Prozentformatierung, es sind Prozentpunkte)
left  = min(-10, float(vals.min())-2)
right = max( 10, float(vals.max())+2)
ax.set_xlim(left, right)
ax.axvline(0, color="grey", linewidth=1)
plt.tight_layout()
footer("Anmerkung: Δ in Prozentpunkten (pp). Positiv = Wachstum, Negativ = Rückgang. Quelle: Statista (2021, 2022, 2024).")
save_fig(FIG / "03_delta_2021_2024_ranking.png")


In [ ]:
# 03_reporting — Cell 7
import numpy as np

order = delta.index  # Reihenfolge nach Δ 2021→2024
y = np.arange(len(order))

fig, ax = plt.subplots(figsize=(10, 10))
x1 = stat_pivot.loc[order, 2021].values
x3 = stat_pivot.loc[order, 2024].values

# Linien zwischen 2021 und 2024
for yi, a, b in zip(y, x1, x3):
    ax.plot([a, b], [yi, yi], marker="o")

ax.set_yticks(y, order)
ax.set_title("Entwicklung 2021→2024 je Kategorie (Dumbbell)")
ax.set_xlabel("Anteil in %")
percent_axis(ax, axis="x", decimals=0, limit=(0, 100))
plt.tight_layout()
footer("Quelle: Statista (2021, 2024). Einheit: Prozent. Hinweis: 2022 im Diagramm nicht dargestellt; siehe Gruppenbalken.")
save_fig(FIG / "03_dumbbell_2021_2024.png")


In [ ]:
# 03_reporting — Cell 8
# Einteilung (anpassbar; bitte fachlich prüfen)
NEEDS = {
    "Lebensmittel / Getränke",
    "Medikamente / Drogerieartikel",
}
LUXURY = {
    "Kleidung / Schuhe",
    "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Bücher / Medien / Software",
    "Hobby- & Freizeitartikel",
    "Möbel / Wohnaccessoires",
}

# Prüfung, ob alle KANON-Kategorien zugeordnet sind
unknown = set(KANON) - NEEDS - LUXURY
if unknown:
    print("Hinweis: nicht zugeordnet (werden ignoriert):", unknown)

def group_share(df_wide: pd.DataFrame, group: set[str]) -> pd.Series:
    common = [c for c in df_wide.index if c in group]
    return df_wide.loc[common, YEARS].sum()

needs_share = group_share(stat_pivot, NEEDS)
lux_share   = group_share(stat_pivot, LUXURY)

need_lux_df = pd.DataFrame({"Bedarfsgüter": needs_share, "Luxusgüter": lux_share}, index=YEARS)
display(need_lux_df)

# Plot: Gruppenvergleich über die Jahre
fig, ax = plt.subplots(figsize=(8, 5))
need_lux_df.plot(kind="bar", ax=ax, width=0.8)
label_bars(ax, decimals=1, rotate=90)
ax.set_title("Anteile Bedarfsgüter vs. Luxusgüter (Summe der Kategorien)")
ax.set_xlabel("Jahr")
ax.set_ylabel("Anteil in %")
percent_axis(ax, axis="y", decimals=0, limit=(0, 100))
ax.legend(title="", frameon=False)
plt.tight_layout()
footer("Hinweis: Einteilung heuristisch; bitte im Methodenteil begründen/validieren. Quelle: Statista (2021–2024).")
save_fig(FIG / "03_needs_vs_luxury.png")

# Verschiebungen in pp
shift_df = pd.DataFrame({
    "Δ Bedarf (2021→2024)": [needs_share[2024] - needs_share[2021]],
    "Δ Luxus (2021→2024)":  [lux_share[2024]  - lux_share[2021]]
})
display(shift_df.style.format("{:+.1f} pp"))


In [ ]:
# 03_reporting — Cell 9
def top_changes(delta_df: pd.DataFrame, k=3):
    inc = delta_df["Δ 2021→2024"].nlargest(k)
    dec = delta_df["Δ 2021→2024"].nsmallest(k)
    return inc, dec

inc, dec = top_changes(delta, k=3)

def bullets(series) -> list[str]:
    return [f"- {idx}: {val:+.1f} pp (2021→2024)" for idx, val in series.items()]

answers = []

# Q1: Welche Produktkategorien wurden mehr oder weniger gekauft?
answers.append("Frage: **Welche Produktkategorien wurden mehr oder weniger gekauft?**")
answers.append("Mehr gekauft (Top-3, Δ in Prozentpunkten 2021→2024):\n" + "\n".join(bullets(inc)))
answers.append("Weniger gekauft (Top-3, Δ in Prozentpunkten 2021→2024):\n" + "\n".join(bullets(dec)))

# Q2: In welchen Warengruppen gab es Wachstum oder Rückgang?
answers.append("Frage: **In welchen Warengruppen gab es Wachstum oder Rückgang?**")
wachs = delta[delta["Δ 2021→2024"] > 0].index.tolist()
rueck = delta[delta["Δ 2021→2024"] < 0].index.tolist()
answers.append("Wachstum: " + (", ".join(wachs) if wachs else "–"))
answers.append("Rückgang: " + (", ".join(rueck) if rueck else "–"))

# Q3: Gibt es Verschiebungen zwischen Bedarfsgütern und Luxusgütern?
answers.append("Frage: **Gibt es Verschiebungen zwischen Bedarfsgütern und Luxusgütern?**")
answers.append(f"- Bedarfsgüter gesamt: 2021 = {needs_share[2021]:.1f} %, 2024 = {needs_share[2024]:.1f} % "
               f"→ Δ = {needs_share[2024]-needs_share[2021]:+.1f} pp")
answers.append(f"- Luxusgüter gesamt:  2021 = {lux_share[2021]:.1f} %, 2024 = {lux_share[2024]:.1f} % "
               f"→ Δ = {lux_share[2024]-lux_share[2021]:+.1f} pp")
answers.append("Interpretation: Positive Δ = Anteil hat zugenommen; negative Δ = Anteil hat abgenommen. "
               "Die Einteilung Bedarf/Luxus ist heuristisch; im Methodenteil kurz begründen (vgl. Vorlesungsleitlinien).")

print("\n\n".join(answers))


In [ ]:
# 03_reporting — Cell 10
REPORT_TXT = OUT / "reporting_kurzbefund.txt"
with open(REPORT_TXT, "w", encoding="utf-8") as f:
    f.write("\n\n".join(answers))
print("Kurzbefund exportiert →", REPORT_TXT)
print("Fertige Abbildungen →", FIG)
